# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [4]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [5]:
from sklearn.cluster import KMeans

In [6]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [8]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [9]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [10]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [24]:
train_data = coordinates.to_numpy()[:100000,:]
cluster_predict = myKMeans.predict(train_data)
centers=myKMeans.cluster_centers_

In [25]:
print((cluster_predict.dtype))
print(train_data.shape)
print(centers.shape)

int32
(100000, 4)
(100, 4)


In [37]:
def show_cluster(cluster_number):
    mask = (cluster_predict==cluster_number)
    cluster_data = train_data[mask]
    
    cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    
    folium.CircleMarker([centers[cluster_number,0], centers[cluster_number,1]], radius=5,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[cluster_number,2], centers[cluster_number,3]], radius=5,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
        
    for i in range(len(cluster_data)):
        folium.CircleMarker([cluster_data[i,0], cluster_data[i,1]], radius=1,                
                        color="yellow", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
        folium.CircleMarker([cluster_data[i,2], cluster_data[i,3]], radius=1,                
                        color="blue", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
                   
    return cluster_map

In [55]:
show_cluster(0)

## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [42]:
import scipy

In [60]:
def cluster_var(cluster_number):
    #Low intra cluster variance
    mask = (cluster_predict==cluster_number)
    cluster_data = train_data[mask]
    
    print(cluster_data)
    D = scipy.spatial.distance.pdist(cluster_data.T, metric='cosine')
    print(D)
    DSum = np.sum(D)
    print(D.shape)
    icv = 1/(len(cluster_data)-1) * DSum
    return icv

In [61]:
cluster_var(0)

[[ 40.73776    -73.98102     40.730497   -73.980668  ]
 [ 40.736011   -73.98543     40.724802   -73.982501  ]
 [ 40.722588   -73.983052    40.733738   -73.988022  ]
 ...
 [ 40.72434    -73.984707    40.726913   -73.989256  ]
 [ 40.718343   -73.987199    40.729908   -73.989067  ]
 [ 40.73547363 -73.99422455  40.72145844 -73.98082733]]
[1.99999998e+00 2.02313296e-08 1.99999999e+00 1.99999998e+00
 7.43220863e-09 1.99999998e+00]
(6,)


0.002954209735886191